## 1. Programmatically using WatsonX.ai models

In [26]:
!pip install langchain==0.0.312
!pip install python-dotenv==1.0.0
!pip install ibm-watson==7.0.1
!pip install ibm-watson-machine-learning==1.0.327
!pip install chromadb==0.4.2
!pip install pypdf==3.12.2
!pip install sentence-transformers==2.2.2
!pip install transformers==4.34.0
!pip install flask-sqlalchemy
!pip install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.0 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 41.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125940 sha256=e95d4075805b9c9fd02e59324709d5e2aa5a5399b0f1e6b3c02750a681887b3d
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.8 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1


    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


In [3]:
# First import the dependencies we need:
import os
from dotenv import load_dotenv
from time import sleep
try:
    from langchain import PromptTemplate
    from langchain.chains import LLMChain, SimpleSequentialChain
    from langchain.document_loaders import PyPDFLoader
    from langchain.indexes import VectorstoreIndexCreator # Vectorize db index with chromadb
    from langchain.embeddings import HuggingFaceEmbeddings # For using HuggingFace embedding models
    from langchain.text_splitter import CharacterTextSplitter # Text splitter

    from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
    from ibm_watson_machine_learning.foundation_models import Model
    from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
    import wget
except ImportError as e:
    print(e)

print("Done importing dependencies.")


Done importing dependencies.


In [8]:
# Get our API key and URL from .env
load_dotenv()
api_key = ''
ibm_cloud_url = 'https://us-south.ml.cloud.ibm.com'
project_id = os.getenv("PROJECT_ID", None)

if api_key is None or ibm_cloud_url is None or project_id is None:
    raise Exception("One or more environment variables are missing!")
else:
    creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }
print("Done getting env variables.")


Done getting env variables.


In [9]:
# Initialize the WatsonX model

# parameters = {
#     "decoding_method": "sample",
#     "min_new_tokens": 1,
#     "max_new_tokens": 100,
#     "temperature": 0.2,
#     "top_k": 25,
#     "top_p": 1,
#     "repetition_penalty": 1.0
# }

params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 25,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 20
}

llm_model = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
)
print("Done initializing LLM.")


Done initializing LLM.


In [10]:
# Predict with the model
countries = ["France", "Japan", "Australia"]

try:
  for country in countries:
    question = f"What is the capital of {country}"
    res = llm_model.generate_text(question)
    print(f"The capital of {country} is {res.capitalize()}")
except Exception as e:
  print(e)


The capital of France is Paris
The capital of Japan is Tokyo
The capital of Australia is Canberra


## 2. Prompt Templates & Chains

In the previous example, the user input is sent directly to the Watsonx LLM, without using Langchain. This is a basic use case, but real applications are rarely so simple. When using an LLM in an application, you will usually need to reuse the same prompt across multiple scenarios. We will now replicate the previous example, but use an LLM chain. This allows us to:

- Accept user input and contruct a prompt
- Generate multiple prompts from a collection of data points in a dataset 

In [11]:
# Define the prompt template
prompt = PromptTemplate(
  input_variables=["country"],
  template= "What is the capital of {country}?",
)

try:
  # In order to use Langchain, we need to instantiate Langchain extension
  lc_llm_model = WatsonxLLM(model=llm_model)
  
  # Define a chain based on model and prompt
  chain = LLMChain(llm=lc_llm_model, prompt=prompt)

  # Getting predictions
  countries = ["Sweden", "Mexico", "Vietnam"]
  for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response.capitalize())
    sleep(0.5)
except Exception as e:
  print(e)


What is the capital of Sweden? = Stockholm
What is the capital of Mexico? = Mexico city
What is the capital of Vietnam? = Hanoi


## 3. Simple sequential chains
The utility of LangChain becomes apparent as we chain outputs of one model as input to another model. Here's a simple example where one generates a question which the other model answers.

LangChain determines a model's output based on its response.  In our examples, the first model creates a response to the end prompt of "Question:" which LangChain maps as an input variable called "question" which it passes to the 2nd model.

In [12]:
# Create two sequential prompts 
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)
print("Done.")


Done.


In [13]:
# Instantiate 2 models (Note, these could be different models depending on use case)
# Note the .to_langchain() method which returns a WatsonxLLM wrapper, like above.
model_1 = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()
model_2 = Model(
    model_id="google/flan-ul2",
    credentials=creds,
    project_id=project_id
).to_langchain()

print("Done.")


Done.


In [14]:
# Construct the sequential chain
prompt_to_model_1 = LLMChain(llm=model_1, prompt=pt1)
prompt_to_model_2 = LLMChain(llm=model_2, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_model_1, prompt_to_model_2], verbose=True)
print("Done.")


Done.


In [15]:
# Run our chain with the topic: "an animal"
# Play around with providing different topics to see the output. eg. cars, the Roman empire
try:
  qa.run("an animal")
except Exception as e:
  print(e)




> Entering new SimpleSequentialChain chain...
What is the most common bird in the world?
pigeon

> Finished chain.


## 4. Easy Loading of Documents Using Lang Chain
LangChain makes it easy to extract passages from documents so that you can answer questions based on your document's content. First download the example PDF file to your working folder: [what is generative ai.pdf](https://github.com/ibm-build-lab/VAD-VAR-Workshop/blob/main/content/Watsonx/WatsonxAI/105/what%20is%20generative%20ai.pdf)

In [22]:
#!wget https://vest-watsonx.s3.us-south.cloud-object-storage.appdomain.cloud/what%20is%20generative%20ai.pdf
!ls

'what is generative ai.pdf'


In [5]:
# Load PDF document
pdf='what is generative ai.pdf'

loaders = [PyPDFLoader(pdf)]
print("Done.")


Done.


In [6]:
# Index loaded PDF
index = VectorstoreIndexCreator(
    embedding=HuggingFaceEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)).from_loaders(loaders)
print("Done.")


Done.


In [9]:
# Initialize watsonx google/flan-ul2 model
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.2,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 100,
    GenParams.MIN_NEW_TOKENS: 50,
    GenParams.MAX_NEW_TOKENS: 300
}
model = Model(
    model_id="google/flan-ul2",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

print("Done.")


Done.


In [10]:
# Init RAG chain
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=model, 
                                    chain_type="stuff", 
                                    retriever=index.vectorstore.as_retriever(), 
                                    input_key="question")
print("Done.")


Done.


In [11]:
# Answer based on the document
res = chain.run("what is Machine Learning?")
print(res)


Machine learning is a type of artificial intelligence. Through machine learning, practitioners develop artificial intelligence through models that can “learn” from data patterns without human direction. The unmanageably huge volume and complexity of data (unmanageable by humans, anyway) that is now being generated has increased the potential of machine learning, as well as the need for it.


Retrieval Augmented Generation (RAG) is a common AI use case. Many companies have vast amounts of data about which they want an AI system to answer questions, do searches or perform summarization tasks. We will learn more about RAG in lab 106.